<a href="https://colab.research.google.com/github/Shivukumarmh/VAAYU-Drug-Discovery-and-inventory-Management/blob/main/Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# Load the descriptor file
file_path = '/content/des_list.csv'  # Replace with your actual file path
descriptor_data = pd.read_csv(file_path)



# Scale the descriptor data to [0, 1] range for stable GAN training
scaler = MinMaxScaler()
descriptor_scaled = scaler.fit_transform(descriptor_data)

# Define input/output dimensions for the GAN based on the descriptor data
input_dim = descriptor_scaled.shape[1]
latent_dim = 100  # Dimension of the random noise for the generator

# Generator model
def build_generator(latent_dim, output_dim):
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_dim=latent_dim),
        layers.Dense(256, activation='relu'),
        layers.Dense(output_dim, activation='sigmoid')  # Sigmoid to generate values between 0 and 1
    ])
    return model

# Discriminator model
def build_discriminator(input_dim):
    model = tf.keras.Sequential([
        layers.Dense(256, activation='relu', input_dim=input_dim),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Binary classification: Real or Fake
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build generator and discriminator
generator = build_generator(latent_dim, input_dim)
discriminator = build_discriminator(input_dim)

# GAN Model: Discriminator is frozen during generator training
discriminator.trainable = False
gan_input = layers.Input(shape=(latent_dim,))
generated_data = generator(gan_input)
gan_output = discriminator(generated_data)
gan_model = tf.keras.Model(gan_input, gan_output)

# Compile the GAN model
gan_model.compile(optimizer='adam', loss='binary_crossentropy')

# Summarize the models
generator.summary()
discriminator.summary()

# Training the GAN
from tensorflow.keras.callbacks import EarlyStopping


# Define EarlyStopping with mode='min' to minimize the generator loss (g_loss)
early_stopping = EarlyStopping(monitor='g_loss', patience=10, restore_best_weights=True, mode='min')


# Early stopping implementation in the training loop
def train_gan_with_manual_early_stopping(generator, discriminator, gan_model, data, epochs=1000, batch_size=64, patience=10):
    batch_count = data.shape[0] // batch_size
    best_loss = np.inf
    patience_counter = 0
    best_weights = None

    for epoch in range(epochs):
        for _ in range(batch_count):
            # Generate random noise for the generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            generated_molecules = generator.predict(noise)

            # Select a batch of real molecules from the dataset
            real_molecules = data[np.random.randint(0, data.shape[0], batch_size)]

            # Train discriminator on real and fake molecules
            d_loss_real = discriminator.train_on_batch(real_molecules, np.ones((batch_size, 1)))
            d_loss_fake = discriminator.train_on_batch(generated_molecules, np.zeros((batch_size, 1)))

            # Train generator and extract the loss value
            g_loss = gan_model.train_on_batch(noise, np.ones((batch_size, 1)))
            g_loss_value = g_loss if isinstance(g_loss, float) else g_loss[0]

        print(f'Epoch {epoch+1}/{epochs} - Generator Loss: {g_loss_value}')

        # Early stopping logic
        if g_loss_value < best_loss:
            best_loss = g_loss_value
            best_weights = generator.get_weights()  # Save the best weights
            patience_counter = 0  # Reset the patience counter
        else:
            patience_counter += 1

        # Stop training if patience exceeded
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    # Restore the best weights after training is done
    if best_weights is not None:
        generator.set_weights(best_weights)
        print(f"Best generator weights restored from epoch with loss: {best_loss}")

# Call the updated training function
train_gan_with_manual_early_stopping(generator, discriminator, gan_model, descriptor_scaled, epochs=1000, batch_size=32, patience=10)
# Generate new molecular descriptors
def generate_new_molecules(generator, num_samples=10, latent_dim=100):
    random_noise = np.random.normal(0, 1, (num_samples, latent_dim))
    generated_molecules = generator.predict(random_noise)
    generated_molecules_rescaled = scaler.inverse_transform(generated_molecules)
    return generated_molecules_rescaled

# Generate 10 new molecular descriptors
new_molecules = generate_new_molecules(generator, num_samples=10)
print(new_molecules)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │          12,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 260)                 │          66,820 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 112,772 (440.52 KB)

 Trainable params: 112,772 (440.52 KB)

...

 Non-trainable params: 99,841 (390.00 KB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━

In [ ]:
# Save the generator model
generator.save('generator_model1.h5')

# Save the discriminator model
discriminator.save('discriminator_model1.h5')

# Save the full GAN model (optional, since you might mainly need the generator)
gan_model.save('gan_model1.h5')